# Oncodriveclustl

In [ ]:
%matplotlib inline

import gzip
from collections import defaultdict

import numpy as np
from intervaltree import IntervalTree
from scipy.signal import argrelmax
import math as m

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from oncodcl_funct import *

### Parse regions

In [ ]:
input_regions = '/home/carnedo/projects/oncodriveclustl/inputs/regions/02_cds.regions.gz'

In [ ]:
trees, regions_d = regions(input_regions)

### Read mutations file and intersect with regions

In [ ]:
input_mutations = '/home/carnedo/projects/oncodriveclustl/inputs/mutations/pancanatlas/BLCA.txt'

In [ ]:
mutations_d = read_mutations(input_mutations, trees)

### Smooth

In [ ]:
total_regions = defaultdict(dict)

for gene in regions_d.keys():
    # Analyze only genes with mutations >= 3
    if len(mutations_d[gene]) >= 3:
        region_info = smoothing(symbol=gene, regions=regions_d, mutations=mutations_d[gene], window=50)
        # dictionary with regions of all genes
        total_regions[region_info['symbol']] = region_info

len(total_regions.keys())

In [ ]:
# Plot results
plt.figure(figsize=(20, 7.5))
ax1 = plt.subplot2grid((1, 5), (0, 0), colspan=4)

ax1.plot(total_regions['TP53']['binary'], c='cornflowerblue')
ax1.set_ylabel('score after smooth', color='cornflowerblue')

ax2 = ax1.twinx()
ax2.plot(total_regions['TP53']['mutations'], c='red')
ax2.set_ylabel('n mutations', color='red')

length = len(total_regions['TP53']['genomic'])

ax1.tick_params('y', colors='cornflowerblue')
ax2.tick_params('y', colors='red')

## Clusters

- Find local maximum and minimum after smoothing for whole cds, according to smoothing score
- Define raw clusters: min(left), max, min(right) ----> Add filter? (ex. clusters wit max > smoothing score of 1 mutation)
- Starting from first maximum in the sequence, search for other relative maximum close to its min(right) border. 
- If there is one, check which of both maximum has a higher smoothing score and merge clusters by updating borders of the highest maximum. The lowest maximum (cluster) is removed from the dictionary of clusters. Iterate through all clusters until no updates are observed. 
- Score clusters. Two scores implemented: using position of smoothing maximum or mutation maximum within the cluster. 

In [ ]:
total_clusters = defaultdict(dict)

for gene in total_regions.keys():
    clusters = clustering(regions=total_regions[gene], mutations=mutations_d[gene], window=25)
    total_clusters[gene] = clusters
    
print(len(total_clusters.keys()))

In [ ]:
# plot clusters

gene = 'TP53'

plot_max = np.zeros(len(total_regions[gene]['binary']))
plot_min_l = np.zeros(len(total_regions[gene]['binary']))
plot_min_r = np.zeros(len(total_regions[gene]['binary']))

for key, value in total_clusters[gene].items(): 
    plot_max[value['max'][0]]+=1
    plot_min_l[value['min_l'][0]]+=1
    plot_min_r[value['min_r'][0]]+=1

    # Change 0 to nan for matplotlib        
plot_max[ plot_max==0.0 ] = np.nan
plot_min_l[ plot_min_l==0.0 ] = np.nan 
plot_min_r[ plot_min_r==0.0 ] = np.nan

plt.figure(figsize=(20, 5))
ax1 = plt.subplot2grid((1, 5), (0, 0), colspan=4)

ax1.plot(total_regions[gene]['binary'], c='cornflowerblue')
ax1.set_ylabel('smoothing score', color='cornflowerblue')

ax2 = ax1.twinx()
ax2.plot(total_regions[gene]['mutations'], c='red')
ax2.set_ylabel('n mutations', color='red')


ax1.plot(plot_max,'|', ms=20, c='green')
ax1.plot(plot_min_l,'|', ms=20, c='orange')
ax1.plot(plot_min_r,'|', ms=20, c='orange')

length = len(total_regions[gene]['binary'])

ax1.tick_params('y', colors='cornflowerblue')
ax2.tick_params('y', colors='red')

In [ ]:
scores_table = '/home/carnedo/projects/oncodriveclustl/outputs/scores_table_3_50_25.txt'

In [ ]:
#### Print table
with open(scores_table, 'w') as fd:
    fd.write('Gene\tCluster\tMin_l\tMax\tMin_l\tWidth\tMut\tScore\t\n')
    for gene, clusters in sorted(total_clusters.items()): 
        for cluster, values in clusters.items(): 
            if values['min_l'] != [] and values['min_r'] != []: 
                width = abs(values['min_l'][0] - values['min_r'][0])
                fd.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%.4f\n" %(gene, cluster, values['min_l'][0], values['max'][0], values['min_r'][0], width, len(mutations_d[gene]), values['score']))
            elif values['min_l'] == []: 
                width = abs(values['max'][0] - values['min_r'][0])
                fd.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%.4f\n" %(gene, cluster, values['min_l'], values['max'][0], values['min_r'][0], width, len(mutations_d[gene]), values['score']))
            elif values['min_r'] == []: 
                width = abs(values['min_l'][0] - values['max'][0])
                fd.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%.4f\n" %(gene, cluster, values['min_l'][0], values['max'][0], values['min_r'], width, len(mutations_d[gene]), values['score']))
